# Analysis Notebook: Direct SQL with In-Memory Temporary Tables

This notebook demonstrates how to perform a multi-step analysis when you only have read-only database access. 

1.  We will run an initial SQL query to fetch the base data.
2.  We will store the results in a pandas DataFrame, which will act as our in-memory temporary table.
3.  We will then run a second query on this DataFrame to perform further analysis.

### Step 1: Setup and Connection

First, we import the necessary libraries and use our helper function to get a direct `pyodbc` connection to the database.

In [1]:
import pandas as pd
import pyodbc
from database_connections import get_db_connection

# Get a connection to the 'v12live' DSN
connection = get_db_connection('v12live')

✅ Connection successful to 'v12live' (v12Live).


### Step 2: Run Initial Query and Create T_A00

We define and execute our main SQL query. The results are loaded into a pandas DataFrame called `T_A00`. This DataFrame is now our temporary table, stored locally in the notebook's memory.

In [2]:
T_A00 = pd.DataFrame()

sql_query = """
SELECT 
    T2.MBWHLO,
    T1.MMSTAT,
    T1.MMCHCD,
    T1.MMINDI,
    T2.MBSTAT,
    T2.MBRESP,
    T2.MBPUIT,
    T1.MMACRF,
    T1.MMITNO AS POPRNO,
    T1.MMITDS,
    T1.MMFUDS,
    T1.MMCFI3 AS PLC
FROM 
    MVXCDTA.MITMAS AS T1 
INNER JOIN 
    MVXCDTA.MITBAL AS T2 ON T1.MMITNO = T2.MBITNO AND T1.MMCONO = T2.MBCONO
WHERE 
    T2.MBWHLO = 'MF1' 
    AND T1.MMSTAT < '80' 
    AND T2.MBSTAT < '80' 
    AND T1.MMACRF <> 'CUSTREP'
    AND T2.MBRESP IN (
        'MP-4210', 'MP-4220', 'MP-4230', 'MP-4250', 'MP-4260', 
        'MP-4270', 'MP-4310', 'MP-4320', 'MP-4330', 'MP-4540'
    )
    AND T2.MBPUIT = 1
"""

if connection:
    try:
        print("--- Fetching initial data from database ---")
        T_A00 = pd.read_sql_query(sql_query, connection)
        
        # FIX: Standardize all column names to lowercase for consistency
        T_A00.columns = T_A00.columns.str.lower()
        
        print(f"✅ Query successful! Found {len(T_A00)} rows.")
        print("Data loaded into 'T_A00' DataFrame.")
        display(T_A00.head())
    except Exception as e:
        print(f"❌ Error executing query: {e}")
    finally:
        connection.close()
        print("\n🔌 Connection closed.")
else:
    print("⚠️ Cannot run query, no active database connection.")

--- Fetching initial data from database ---


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_25348\2059134312.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  T_A00 = pd.read_sql_query(sql_query, connection)


✅ Query successful! Found 835 rows.
Data loaded into 'T_A00' DataFrame.


,mbwhlo,mmstat,mmchcd,mmindi,mbstat,mbresp,mbpuit,mmacrf,poprno,mmitds,mmfuds,plc
0,MF1,20,0.0,3.0,20,MP-4320,1.0,.6S,253994,KCD2-UT2-1,Signal converter KCD2-UT2-1 ...,300
1,MF1,20,0.0,3.0,20,MP-4220,1.0,.4S,814139,PCBA C 5500 DC Pwr Brd Int Mt,Assy Platine 103-1058.. Pwr B...,310
2,MF1,40,0.0,3.0,20,MP-4220,1.0,.4S,471662,PCBA C S890-DI890 Mother BD,BASE MODEL MOTHER BOARD DI890 8Ch ...,490
3,MF1,20,0.0,3.0,20,MP-4320,1.0,.6S,096045,KFA6-ER-1.5,Electrode relay KFA6-ER-1.5 ...,310
4,MF1,20,0.0,3.0,20,MP-4320,1.0,.6S,096046,KFA6-ER-1.6,Electrode relay KFA6-ER-1.6 ...,310



🔌 Connection closed.


### Step 3: Create A01_Varianten_Check

This step performs the equivalent of a `GROUP BY` and `COUNT` to find the number of items per PLC.

In [3]:
if not T_A00.empty:
    print("--- Creating A01_Varianten_Check ---")
    
    # Using lowercase column names now
    A01_Varianten_Check = T_A00.groupby('plc')['poprno'].count().reset_index()
    A01_Varianten_Check = A01_Varianten_Check.rename(columns={'poprno': 'ItemCount'})
    A01_Varianten_Check = A01_Varianten_Check.sort_values(by='ItemCount', ascending=False)
    
    print("✅ Analysis complete.")
    display(A01_Varianten_Check.head(10))
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")

--- Creating A01_Varianten_Check ---
✅ Analysis complete.


,plc,ItemCount
3,310,591
4,311,58
10,520,58
7,490,46
0,200,22
1,240,14
8,491,12
2,300,11
12,INT,9
9,510,7


### Step 4: Create A02_ACO_Check

This cell performs another aggregation on our initial DataFrame to count items based on planner code and the `MMACRF` field.

In [4]:
if not T_A00.empty:
    print("--- Creating A02_ACO_Check ---")
    
    # Perform the groupby and count operation using lowercase names
    A02_ACO_Check = T_A00.groupby(['mbresp', 'mmacrf'])['poprno'].count().reset_index()
    
    # Rename the columns to match the SQL 'AS' clauses
    A02_ACO_Check = A02_ACO_Check.rename(columns={
        'mbresp': 'PlannerCode',
        'mmacrf': 'MMACRF',
        'poprno': 'CntItems'
    })
    
    # Sort the results by MMACRF
    A02_ACO_Check = A02_ACO_Check.sort_values(by='MMACRF')
    
    print("✅ Analysis complete.")
    display(A02_ACO_Check)
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")

--- Creating A02_ACO_Check ---
✅ Analysis complete.


,PlannerCode,MMACRF,CntItems
0,MP-4210,.4S,100
2,MP-4220,.4S,114
15,MP-4330,.4S,6
4,MP-4250,.4S,18
7,MP-4260,.4S,1
17,MP-4540,.4S,42
10,MP-4310,.4S,6
12,MP-4320,.4S,3
5,MP-4250,.6M,1
13,MP-4320,.6N,2


### Step 5: Create A03_LotControled

This step performs a more detailed aggregation, grouping by three columns to count items based on their lot control status (`MMINDI`).

In [5]:
if not T_A00.empty:
    print("--- Creating A03_LotControled ---")
    
    # Perform the groupby operation on three columns using lowercase names
    A03_LotControled = T_A00.groupby(['mbresp', 'mmacrf', 'mmindi'])['poprno'].count().reset_index()
    
    # Rename the columns
    A03_LotControled = A03_LotControled.rename(columns={
        'mbresp': 'PlannerCode',
        'poprno': 'Cnt_LotCrtl'
    })
    
    # Sort the results
    A03_LotControled = A03_LotControled.sort_values(by=['mmacrf', 'mmindi'])
    
    print("✅ Analysis complete.")
    display(A03_LotControled)
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")

--- Creating A03_LotControled ---
✅ Analysis complete.


,PlannerCode,mmacrf,mmindi,Cnt_LotCrtl
0,MP-4210,.4S,0.0,5
5,MP-4220,.4S,0.0,21
12,MP-4260,.4S,0.0,1
15,MP-4310,.4S,0.0,6
17,MP-4320,.4S,0.0,3
1,MP-4210,.4S,1.0,14
6,MP-4220,.4S,1.0,1
2,MP-4210,.4S,3.0,81
7,MP-4220,.4S,3.0,92
9,MP-4250,.4S,3.0,18


### Step 6: Create A03_PLCCheck

This aggregation counts the number of items for each combination of planner and product line code (`PLC`).

In [6]:
if not T_A00.empty:
    print("--- Creating A03_PLCCheck ---")
    
    # Perform the groupby operation using lowercase names
    A03_PLCCheck = T_A00.groupby(['mbresp', 'plc'])['poprno'].count().reset_index()
    
    # Rename the columns
    A03_PLCCheck = A03_PLCCheck.rename(columns={
        'mbresp': 'PlannerCode',
        'poprno': 'CountOfPOPRNO'
    })
    
    # Sort the results
    A03_PLCCheck = A03_PLCCheck.sort_values(by='plc')
    
    print("✅ Analysis complete.")
    display(A03_PLCCheck)
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")

--- Creating A03_PLCCheck ---
✅ Analysis complete.


,PlannerCode,plc,CountOfPOPRNO
0,MP-4210,200,8
19,MP-4260,200,2
16,MP-4250,200,1
8,MP-4220,200,5
41,MP-4540,200,2
36,MP-4330,200,2
26,MP-4270,200,2
20,MP-4260,240,3
1,MP-4210,240,10
37,MP-4330,240,1


### Step 7: Link DataFrame to New Tables and Create T_A10

This step links our initial DataFrame (`T_A00`) with two new database tables (`MPDOPE` and `MPDHED`).

1.  We extract the unique item numbers (`POPRNO`) from our DataFrame.
2.  We use these item numbers to build a new SQL query that efficiently fetches only the required data from the database.
3.  We execute this query and get a new DataFrame with the details.
4.  Finally, we perform a `merge` (join) in pandas to combine our original data with the new details, creating the final `T_A10` DataFrame.

In [7]:
T_A10 = pd.DataFrame() # Initialize T_A10 as an empty DataFrame
if not T_A00.empty:
    print("--- Linking DataFrame to MPDOPE and MPDHED tables ---")
    
    # 1. Extract the unique keys from your DataFrame using the lowercase name
    item_numbers = T_A00['poprno'].unique().tolist()
    
    if item_numbers:
        # 2. Format the list of keys for the SQL 'IN' clause
        formatted_keys = ", ".join([f"'{item}'" for item in item_numbers])

        # 3. Construct the new SQL query
        details_sql_query = f"""
        SELECT 
            T_OPE.POPRNO, T_HED.PHSTAT, T_OPE.POSTRT, T_OPE.POOPNO, T_OPE.POPLGR, 
            T_OPE.POOPDS, T_OPE.POTXT1, T_OPE.POTXT2, T_OPE.PODOID, T_OPE.POAURP, 
            T_OPE.POCONO, T_OPE.POFACI
        FROM 
            MVXCDTA.MPDOPE AS T_OPE
        INNER JOIN 
            MVXCDTA.MPDHED AS T_HED ON T_OPE.POPRNO = T_HED.PHPRNO 
                                    AND T_OPE.POFACI = T_HED.PHFACI 
                                    AND T_OPE.POCONO = T_HED.PHCONO
        WHERE 
            T_OPE.POSTRT = 'STD'
            AND T_OPE.POCONO = 1
            AND T_OPE.POFACI = 'MF1'
            AND T_OPE.POTDAT = 99999999
            AND T_OPE.POPRNO IN ({formatted_keys})
        """

        # 4. Execute the query to get details
        df_details = pd.DataFrame()
        connection = get_db_connection('v12live') # Re-open a connection
        if connection:
            try:
                print(f"Fetching details for {len(item_numbers)} items...")
                df_details = pd.read_sql_query(details_sql_query, connection)
                
                # Standardize details columns to lowercase
                df_details.columns = df_details.columns.str.lower()
                
                print(f"✅ Query successful! Found {len(df_details)} matching detail rows.")

                # 5. Perform the final join in pandas to create T_A10
                T_A10 = pd.merge(
                    left=T_A00,
                    right=df_details,
                    on='poprno', # Join on the lowercase common item number column
                    how='inner'  # Use 'inner' to match the SQL INNER JOIN
                )
                
                # Select and reorder columns, including 'mbresp' for the next step
                T_A10 = T_A10[[
                    'plc', 'poprno', 'mmacrf', 'mbpuit', 'phstat', 'postrt', 
                    'poopno', 'poplgr', 'poopds', 'potxt1', 'potxt2', 'podoid', 'poaurp', 
                    'pocono', 'pofaci', 'mbresp'
                ]]

                print("\n--- Final T_A10 DataFrame created ---")
                display(T_A10.head())

            except Exception as e:
                print(f"❌ Error executing the details query: {e}")
            finally:
                connection.close()
                print("\n🔌 Connection closed.")
    else:
        print("⚠️ No item numbers in T_A00 to use for the next query.")
else:
    print("⚠️ T_A00 DataFrame is empty, skipping the linking step.")

--- Linking DataFrame to MPDOPE and MPDHED tables ---
✅ Connection successful to 'v12live' (v12Live).
Fetching details for 835 items...


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_25348\470727697.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_details = pd.read_sql_query(details_sql_query, connection)


✅ Query successful! Found 28033 matching detail rows.

--- Final T_A10 DataFrame created ---


,plc,poprno,mmacrf,mbpuit,phstat,postrt,poopno,poplgr,poopds,potxt1,potxt2,podoid,poaurp,pocono,pofaci,mbresp
0,300,253994,.6S,1.0,20,STD,10.0,4320 M,MATERIAL KITTING,...,Product transfer from Interface back to Analog...,,1.0,1.0,MF1,MP-4320
1,300,253994,.6S,1.0,20,STD,1000.0,4313 K,KANBAN KITTING INTERFACE PRINT,...,...,,1.0,1.0,MF1,MP-4320
2,300,253994,.6S,1.0,20,STD,1100.0,4324,PRINTING TAMPON 1,PlanTimeYr- PRT-A261 and PRT-A264 (SMT) ...,...,260-5767E,2.0,1.0,MF1,MP-4320
3,300,253994,.6S,1.0,20,STD,1110.0,4320,AID/RESOURCE/FIXTURE,...,...,65-2767A,1.0,1.0,MF1,MP-4320
4,300,253994,.6S,1.0,20,STD,1300.0,4117 K,KANBAN KITTING MATL FORMING,...,...,,1.0,1.0,MF1,MP-4320



🔌 Connection closed.


### Step 8: Group by Planner and Document ID to Create T_A11

This step creates the `T_A11` table by aggregating the `T_A10` data. It trims whitespace from the document ID, groups by planner and the cleaned ID, and counts the number of operations.

In [8]:
T_A11 = pd.DataFrame() # Initialize T_A11 as an empty DataFrame
if not T_A10.empty:
    print("--- Creating T_A11 by grouping T_A10 ---")
    
    # Create a working copy to avoid SettingWithCopyWarning
    temp_df = T_A10.copy()
    
    # Trim whitespace from the 'podoid' column, handling potential non-string data
    temp_df['m3doid'] = temp_df['podoid'].astype(str).str.strip()
    
    # Perform the groupby operation
    T_A11 = temp_df.groupby(['mbresp', 'm3doid'])['poopno'].count().reset_index()
    
    # Rename the columns to match the desired output
    T_A11 = T_A11.rename(columns={
        'mbresp': 'PlannerCode',
        'm3doid': 'M3DOID',
        'poopno': 'UsageCnt'
    })
    
    print("✅ T_A11 DataFrame created successfully.")
    display(T_A11.head())
else:
    print("⚠️ T_A10 DataFrame is empty, skipping creation of T_A11.")

--- Creating T_A11 by grouping T_A10 ---
✅ T_A11 DataFrame created successfully.


,PlannerCode,M3DOID,UsageCnt
0,MP-4210,,816
1,MP-4210,01-6463B,1
2,MP-4210,01-6760B,1
3,MP-4210,01-6785D,4
4,MP-4210,01-6800D,2


### Step 9: Join with EDW Database to create A12_DoKID_EDM
This step connects to a different database (EDMEWAREAD) to enrich our T_A11 data.

1. We extract the unique document IDs (M3DOID) from T_A11.
2. We connect to the edw DSN.
3. We query the ADMEDP_EDM_DOCS table using these IDs, breaking them into chunks to avoid database errors.
4. We perform a final merge (join) in pandas to combine the usage counts with the document details.

In [17]:
A12_DoKID_EDM = pd.DataFrame() # Initialize as empty
if not T_A11.empty:
    print("--- Joining T_A11 with ADMEDP_EDM_DOCS from EDW database ---")
    
    # 1. Extract the unique document IDs from T_A11 and filter out empty strings
    doc_ids = [doc_id for doc_id in T_A11['M3DOID'].unique().tolist() if doc_id]
    
    if doc_ids:
        # 2. Connect to the 'edw' database
        edw_connection = get_db_connection('edw')
        
        if edw_connection:
            df_docs_details_list = []
            # FIX: Break the list of IDs into chunks of 999 to avoid Oracle DB error
            chunk_size = 999
            id_chunks = [doc_ids[i:i + chunk_size] for i in range(0, len(doc_ids), chunk_size)]
            
            try:
                for i, chunk in enumerate(id_chunks):
                    print(f"Fetching document details from EDW: chunk {i+1}/{len(id_chunks)}")
                    formatted_doc_ids = ", ".join([f"'{doc_id}'" for doc_id in chunk])
                    
                    # 3. Construct the query for the EDW database
                    docs_sql_query = f"""
                    SELECT 
                        DOCNUMBER, 
                        INFO, 
                        INFO1
                    FROM 
                        ADMEDP.EDM_DOCS
                    WHERE 
                        DOCNUMBER IN ({formatted_doc_ids})
                    """
                    df_chunk = pd.read_sql_query(docs_sql_query, edw_connection)
                    df_docs_details_list.append(df_chunk)
                
                # Combine all chunks into one DataFrame
                df_docs_details = pd.concat(df_docs_details_list, ignore_index=True)
                df_docs_details.columns = df_docs_details.columns.str.lower()
                print(f"✅ Query successful! Found {len(df_docs_details)} matching documents in total.")
                
                # 4. Join the T_A11 data with the document details
                A12_DoKID_EDM = pd.merge(
                    left=T_A11,
                    right=df_docs_details,
                    left_on='M3DOID',
                    right_on='docnumber',
                    how='inner'
                )
                
                # Sort the final results
                A12_DoKID_EDM = A12_DoKID_EDM.sort_values(by='M3DOID')
                
                print("\n--- Final Enriched Data --- ")
                display(A12_DoKID_EDM.head())

            except Exception as e:
                print(f"❌ Error executing EDW query: {e}")
            finally:
                edw_connection.close()
                print("\n🔌 EDW Connection closed.")
else:
    print("⚠️ T_A11 DataFrame is empty, skipping final join.")

--- Joining T_A11 with ADMEDP_EDM_DOCS from EDW database ---
✅ Connection successful to 'edw' (EDMEWAREAD).
Fetching document details from EDW: chunk 1/6


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_25348\700660451.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(docs_sql_query, edw_connection)


Fetching document details from EDW: chunk 2/6
Fetching document details from EDW: chunk 3/6
Fetching document details from EDW: chunk 4/6
Fetching document details from EDW: chunk 5/6
Fetching document details from EDW: chunk 6/6
✅ Query successful! Found 5330 matching documents in total.

--- Final Enriched Data --- 


,PlannerCode,M3DOID,UsageCnt,docnumber,info,info1
3867,MP-4320,01-5616,2,01-5616,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627
4612,MP-4540,01-6366,15,01-6366,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel
4613,MP-4540,01-6367,7,01-6367,schematic to ISTA-2**-BP**-R redundant baseplate,02.09.02 PA-EW A. Hartnagel
4614,MP-4540,01-6368A,1,01-6368A,schematic to PCBA T ISTA-BP MATRIX 1S (for FBM...,None
4615,MP-4540,01-6369A,1,01-6369A,schematic to PCBA T ISTA-BP MATRIX 2S (for FBM...,None



🔌 EDW Connection closed.


### Step 10: Join with EDW File Details to create A13_Doc_type_type

This step enriches the data further by connecting to the `EDMEWAREAD` database again to get file-specific details from the `ADMEDP_EDM_FILES` table.

1.  We extract the unique document IDs (`M3DOID`) from `A12_DoKID_EDM`.
2.  We query the `ADMEDP_EDM_FILES` table for matching documents.
3.  We join the results and then create two new columns: `FileExt` by extracting the file extension from the filename, and a hardcoded `Ausdr1` column.

In [18]:
if not A12_DoKID_EDM.empty:
    print("--- Joining with ADMEDP_EDM_FILES from EDW database ---")
    
    # 1. Extract the unique document IDs from A12_DoKID_EDM
    doc_ids = A12_DoKID_EDM['M3DOID'].unique().tolist()
    
    if doc_ids:
        # 2. Connect to the 'edw' database
        edw_connection = get_db_connection('edw')
        
        if edw_connection:
            df_files_details_list = []
            # FIX: Break the list of IDs into chunks of 999
            chunk_size = 999
            id_chunks = [doc_ids[i:i + chunk_size] for i in range(0, len(doc_ids), chunk_size)]
            
            try:
                for i, chunk in enumerate(id_chunks):
                    print(f"Fetching file details from EDW: chunk {i+1}/{len(id_chunks)}")
                    formatted_doc_ids = ", ".join([f"'{doc_id}'" for doc_id in chunk])
                    
                    # Construct the query for the EDW database
                    files_sql_query = f"""
                    SELECT 
                        DOCNUMBER, 
                        FILENAME, 
                        FILEUSER
                    FROM 
                        ADMEDP.EDM_FILES
                    WHERE 
                        DOCNUMBER IN ({formatted_doc_ids})
                    """
                    df_chunk = pd.read_sql_query(files_sql_query, edw_connection)
                    df_files_details_list.append(df_chunk)

                # Combine all chunks into one DataFrame
                df_files_details = pd.concat(df_files_details_list, ignore_index=True)
                df_files_details.columns = df_files_details.columns.str.lower()
                print(f"✅ Query successful! Found {len(df_files_details)} matching files.")
                
                # Join the A12 data with the file details
                merged_df = pd.merge(
                    left=A12_DoKID_EDM,
                    right=df_files_details,
                    left_on='M3DOID',
                    right_on='docnumber',
                    how='inner'
                )
                
                # 3. Create the new columns
                # Replicate: Mid([FILENAME],InStr(1,Trim([FILENAME]),".")+1,3)
                merged_df['FileExt'] = merged_df['filename'].str.strip().str.split('.').str[-1].str.slice(0, 3)
                
                # Replicate: "X" AS Ausdr1
                merged_df['Ausdr1'] = 'X'
                
                # Select and reorder final columns
                A13_Doc_type_type = merged_df[[
                    'PlannerCode', 'M3DOID', 'UsageCnt', 'docnumber_x', 'info', 'info1', 
                    'FileExt', 'fileuser', 'Ausdr1'
                ]]
                # Rename the duplicated docnumber column for clarity
                A13_Doc_type_type = A13_Doc_type_type.rename(columns={'docnumber_x': 'DOCNUMBER'})
                
                print("\n--- Final A13_Doc_type_type DataFrame Created ---")
                display(A13_Doc_type_type.head())

            except Exception as e:
                print(f"❌ Error executing EDW query: {e}")
            finally:
                edw_connection.close()
                print("\n🔌 EDW Connection closed.")
else:
    print("⚠️ A12_DoKID_EDM DataFrame is empty, skipping final join.")

--- Joining with ADMEDP_EDM_FILES from EDW database ---
✅ Connection successful to 'edw' (EDMEWAREAD).
Fetching file details from EDW: chunk 1/6


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_25348\2989723795.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(files_sql_query, edw_connection)


Fetching file details from EDW: chunk 2/6
Fetching file details from EDW: chunk 3/6
Fetching file details from EDW: chunk 4/6
Fetching file details from EDW: chunk 5/6
Fetching file details from EDW: chunk 6/6
✅ Query successful! Found 12380 matching files.

--- Final A13_Doc_type_type DataFrame Created ---


,PlannerCode,M3DOID,UsageCnt,DOCNUMBER,info,info1,FileExt,fileuser,Ausdr1
0,MP-4320,01-5616,2,01-5616,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,pdf,282.0,X
1,MP-4320,01-5616,2,01-5616,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,sch,282.0,X
2,MP-4320,01-5616,2,01-5616,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,txt,282.0,X
3,MP-4540,01-6366,15,01-6366,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel,txt,282.0,X
4,MP-4540,01-6366,15,01-6366,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel,sch,282.0,X



🔌 EDW Connection closed.


### Step 11: Create Pivot Table A14_Doc-type
This step replicates the MS Access TRANSFORM...PIVOT query. We use the pandas pivot_table function to transform the A13_Doc_type_type DataFrame. The unique values from the FileExt column will become the new columns, and the values will be the count of occurrences for each document type.

In [19]:
if not A13_Doc_type_type.empty:
    print("--- Creating pivot table A14_Doc-type ---")
    
    # Define the columns that will remain as rows (the index of the pivot)
    index_cols = [
        'PlannerCode', 
        'M3DOID', 
        'UsageCnt', 
        'DOCNUMBER', 
        'info', 
        'info1', 
        'fileuser'
    ]
    
    # Use pivot_table to replicate the TRANSFORM query
    # We use 'UsageCnt' for the values and count them. Any column could be used for counting.
    A14_Doc_type = pd.pivot_table(
        A13_Doc_type_type,
        values='UsageCnt',      # The values to aggregate (we're just counting)
        index=index_cols,       # Columns to group by on the rows
        columns=['FileExt'],    # Column whose values will become new columns
        aggfunc='count',        # The aggregation function (count occurrences)
        fill_value=0            # Fill any missing combinations with 0
    ).reset_index() # Flatten the pivot table back into a regular DataFrame
    
    print("✅ A14_Doc-type DataFrame created successfully.")
    display(A14_Doc_type.head())
else:
    print("⚠️ A13_Doc_type_type DataFrame is empty, skipping pivot operation.")

--- Creating pivot table A14_Doc-type ---
✅ A14_Doc-type DataFrame created successfully.


FileExt,PlannerCode,M3DOID,UsageCnt,DOCNUMBER,info,info1,fileuser,1,2,3,...,sch,stl,stp,txt,vlm,vsd,xls,zip,zpl,zpo
0,MP-4210,01-6760B,1,01-6760B,"HD2-DM-B, Fieldbus Power Hub Basic Diagnostic ...",HD2-DM-B,716.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,MP-4210,01-7909,1,01-7909,"MB-FB-DM, Stand alone diagnostic module, 202360",MB-FB-DM; Standalone diagnostic module; #202360,1121.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,MP-4210,01-9713D,2,01-9713D,Schematic Drawing - Horizon Project (DDE-2725)...,HMI Zone1,2155.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,MP-4210,01-9799E,2,01-9799E,Schematic for 05-7206D..\nDisplay Unit for Vis...,Mohammad Sawaftah HMI,20543.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MP-4210,03-ACT7F,4,03-ACT7F,b\nAssembly plan for 05-7166D...\nCPU UNIT PCB...,HMI Zone1,2155.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
